# Data Rationalisation

This notebook prepares and inserts data from a `.csv` source into an existing database table.

## Imports

In [76]:
import pandas as pd
from redshift_connector import connect
import re

## Setup

In [77]:
def get_db_connection():
    return connect(host="c17-redshift-cluster.cdq12ms5gjyk.eu-west-2.redshift.amazonaws.com",
                   user="admin",
                   password="Password1",
                   database="dw_air_travel",
                   port=5439)

conn = get_db_connection()

## Data Sourcing

In [84]:
with conn.cursor() as cur:
    cur.execute("select * from ticket.event;")
    events = cur.fetch_dataframe()
    cur.execute("select * from ticket.users;")
    users = cur.fetch_dataframe()
    cur.execute("select * from ticket.sales;")
    sales = cur.fetch_dataframe()


In [85]:
events.head()

,eventid,venueid,catid,dateid,eventname,starttime
0,4135,16,9,1827,Nas,2008-01-01 14:30:00
1,3915,51,9,1827,Return To Forever,2008-01-01 14:00:00
2,5807,45,9,1827,Return To Forever,2008-01-01 15:00:00
3,1217,238,6,1827,Mamma Mia!,2008-01-01 20:00:00
4,6711,72,9,1827,Smashing Pumpkins,2008-01-01 19:00:00


In [90]:
events.set_index("starttime").resample("M").count()

/var/folders/5_/zmdksvlj7s517k3r6w2bcg4h0000gn/T/ipykernel_1251/3150191826.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  events.set_index("starttime").resample("M").count()


,eventid,venueid,catid,dateid,eventname
starttime,,,,,
2008-01-31,777,777,777,777,777
2008-02-29,711,711,711,711,711
2008-03-31,753,753,753,753,753
2008-04-30,725,725,725,725,725
2008-05-31,727,727,727,727,727
2008-06-30,709,709,709,709,709
2008-07-31,729,729,729,729,729
2008-08-31,737,737,737,737,737
2008-09-30,746,746,746,746,746


In [87]:
sales.head()

,salesid,listid,sellerid,buyerid,eventid,dateid,qtysold,pricepaid,commission,saletime
0,150314,173969,48680,816,8762,1827,2,688.00,103.20,2008-01-01 03:50:02
1,33095,36572,30047,660,2903,1827,2,234.00,35.10,2008-01-01 09:41:06
2,157751,206999,3003,157,6605,1827,1,1730.00,259.50,2008-01-01 12:50:55
3,88268,100813,45818,698,8649,1827,4,836.00,125.40,2008-01-01 07:26:20
4,110917,127048,37631,116,1749,1827,1,337.00,50.55,2008-01-01 07:05:02


In [88]:
sales.describe()

,salesid,listid,sellerid,buyerid,eventid,dateid,qtysold,saletime
count,172455.000000,172455.000000,172455.000000,172455.000000,172455.000000,172455.000000,172455.000000,172455
mean,86229.000000,101477.051642,25031.674483,16524.098814,4391.790113,2008.304456,2.002522,2008-07-01 11:10:57.419953152
min,2.000000,4.000000,1.000000,1.000000,1.000000,1827.000000,1.000000,2008-01-01 01:00:19
25%,43115.500000,48411.500000,12518.000000,6606.000000,2182.000000,1926.000000,1.000000,2008-04-10 06:29:05.500000
50%,86229.000000,98447.000000,25106.000000,14407.000000,4380.000000,2009.000000,2.000000,2008-07-02 07:08:32
75%,129342.500000,148335.000000,37583.500000,24744.000000,6583.000000,2090.000000,2.000000,2008-09-21 08:40:27.500000
max,172456.000000,235494.000000,49990.000000,49984.000000,8798.000000,2191.000000,8.000000,2008-12-31 12:58:19
std,49783.614674,62763.447378,14432.158479,11780.933169,2544.097352,95.401862,1.068621,NaN
